In [20]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [21]:
df = pd.read_csv("../../data/csv/anime_data.csv")

In [22]:
df

,Unnamed: 0,title,genre,episodes,popularity,score
0,0,"""0""",['Music'],1.0,7345.0,4.77
1,1,"""Aesop"" no Ohanashi yori: Ushi to Kaeru, Yokub...",['Kids'],1.0,12413.0,5.61
2,2,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi","['Comedy', 'Fantasy', 'School']",1.0,3466.0,6.96
3,3,"""Bungaku Shoujo"" Memoire","['Drama', 'Romance', 'School']",3.0,2943.0,7.40
4,4,"""Bungaku Shoujo"" Movie","['Mystery', 'Drama', 'Romance', 'School']",1.0,1799.0,7.48
...,...,...,...,...,...,...
15868,15868,xxxHOLiC Rou,"['Mystery', 'Supernatural']",2.0,1869.0,8.21
15869,15869,xxxHOLiC Shunmuki,"['Mystery', 'Comedy', 'Psychological', 'Supern...",2.0,1813.0,8.13
15870,15870,Üks Uks,['Dementia'],1.0,14107.0,5.71
15871,15871,ēlDLIVE,"['Action', 'Sci-Fi', 'Space', 'Police', 'Shoun...",12.0,2260.0,6.23


In [23]:
df.drop(columns=["Unnamed: 0"], inplace=True)

In [24]:
df_new = df[df["episodes"] != "Not specified"]

In [25]:
df_new

,title,genre,episodes,popularity,score
0,"""0""",['Music'],1.0,7345.0,4.77
1,"""Aesop"" no Ohanashi yori: Ushi to Kaeru, Yokub...",['Kids'],1.0,12413.0,5.61
2,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi","['Comedy', 'Fantasy', 'School']",1.0,3466.0,6.96
3,"""Bungaku Shoujo"" Memoire","['Drama', 'Romance', 'School']",3.0,2943.0,7.40
4,"""Bungaku Shoujo"" Movie","['Mystery', 'Drama', 'Romance', 'School']",1.0,1799.0,7.48
...,...,...,...,...,...
15868,xxxHOLiC Rou,"['Mystery', 'Supernatural']",2.0,1869.0,8.21
15869,xxxHOLiC Shunmuki,"['Mystery', 'Comedy', 'Psychological', 'Supern...",2.0,1813.0,8.13
15870,Üks Uks,['Dementia'],1.0,14107.0,5.71
15871,ēlDLIVE,"['Action', 'Sci-Fi', 'Space', 'Police', 'Shoun...",12.0,2260.0,6.23


In [26]:
import pandas as pd
import ast

# Convert 'genre' to a list
df_new['genre'] = df_new['genre'].apply(ast.literal_eval)

# Explode the DataFrame
df_exploded = df_new.explode('genre')

In [27]:
# Define the genres and their corresponding broad categories
genre_categories = {
    'Action and Adventure': ['Action', 'Adventure', 'Super Power', 'Martial Arts', 'Samurai', 'Military'],
    'Fantasy and Supernatural': ['Fantasy', 'Supernatural', 'Magic', 'Vampire', 'Demons'],
    'Science Fiction and Technology': ['Sci-Fi', 'Mecha', 'Space', 'Cars'],
    'Comedy and Slice of Life': ['Comedy', 'Slice of Life', 'Parody'],
    'Drama and Romance': ['Drama', 'Romance', 'Shoujo', 'Shounen Ai', 'Josei', 'Shoujo Ai', 'Yaoi'],
    'Horror and Thriller': ['Horror', 'Thriller', 'Psychological', 'Mystery'],
    'Historical and Cultural': ['Historical', 'Samurai', 'Seinen'],
    'Youth and School Life': ['School', 'Shounen', 'Kids', 'Shoujo'],
    'Special Interests': ['Ecchi', 'Harem', 'Hentai', 'Yaoi', 'Yuri'],
    'Miscellaneous': ['Music', 'Sports', 'Game', 'Police', 'Dementia']
}

# Function to map each genre to its broader category
def map_genre_to_category(genre):
    for category, genres in genre_categories.items():
        if genre in genres:
            return category
    return 'Miscellaneous'

# Map each genre to its broader category
df_exploded['category'] = df_exploded['genre'].apply(map_genre_to_category)

In [28]:
df_exploded['title'] = df_exploded['title'].str.replace(r"[^\w\s]", '', regex=True)


In [29]:
df_exploded = df_exploded.groupby(["category", "title"], as_index=False)[["popularity", "score"]].mean()

In [30]:
# Group by genre and sort
top_score = df_exploded.groupby('category', as_index=False).apply(lambda x: x.nlargest(10, 'score'))
top_popularity = df_exploded.groupby('category', as_index=False).apply(lambda x: x.nlargest(10, 'popularity'))

In [31]:
top_score.reset_index(drop=True, inplace=True)

In [32]:
top_score_final = top_score.groupby(["category", "title"], as_index=False)["score"].mean()

In [33]:
top_score_final

,category,title,score
0,Action and Adventure,Code Geass Hangyaku no Lelouch R2,8.93
1,Action and Adventure,Fullmetal Alchemist Brotherhood,9.23
2,Action and Adventure,Ginga Eiyuu Densetsu,9.03
3,Action and Adventure,Gintama,9.00
4,Action and Adventure,Gintama Enchousen,8.98
...,...,...,...
95,Youth and School Life,Hunter x Hunter 2011,9.11
96,Youth and School Life,Kimi no Na wa,9.09
97,Youth and School Life,Koe no Katachi,9.01
98,Youth and School Life,Quiz de Manabu Pinocchio no Koutsuu Ansen,9.08


In [34]:
top_popularity.reset_index(drop=True, inplace=True)

In [35]:
top_popularity_final = top_popularity.groupby(["category", "title"], as_index=False)["popularity"].mean()

In [36]:
top_popularity_final

,category,title,popularity
0,Action and Adventure,Boy General,16308.0
1,Action and Adventure,Jumbagi Hanbandoui gongryong 3D,16238.0
2,Action and Adventure,Mabeob Chunjamun Season 2,16305.0
3,Action and Adventure,Miniforce,16166.0
4,Action and Adventure,Ninja Tamamaru no Hi no Youjin,16225.0
...,...,...,...
95,Youth and School Life,Robot Trains,16315.0
96,Youth and School Life,Usagi no Mofy TV 2014,16317.0
97,Youth and School Life,Usagi no Mofy TV 2016,16319.0
98,Youth and School Life,Watashi no Ningyou,16310.0


In [37]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Create a subplot
fig = make_subplots()

# Group data by category
categories = top_score_final['category'].unique()

# Add a trace for each anime title, making bars horizontal
# Start with the first category visible
first = True
for category in categories:
    category_df = top_score_final[top_score_final['category'] == category]
    category_df.sort_values(by='score', inplace=True)
    fig.add_trace(
        go.Bar(y=category_df['title'], x=category_df['score'], name=category, 
               orientation='h', visible=first, marker_color='#df6200')  # Set bar color to crimson
    )
    first = False  # Set the rest of the categories to invisible

# Create a dropdown menu
buttons = []
for i, category in enumerate(categories):
    button = dict(
        label=category,
        method="update",
        args=[{"visible": [i == j for j in range(len(categories))]},  # Only the selected category is visible
              {"title": f"Top Animes (Score) for Genre: {category}"}])
    buttons.append(button)

# Update layout for dropdown, axes, and xaxis range
fig.update_layout(
    updatemenus=[dict(
        active=0,  # First category is active
        buttons=buttons,
        direction="down",
        pad={"r": 10, "t": 10},
        showactive=True,
        x=0.77,
        xanchor="left",
        y=1.25,
        yanchor="top"
    )],
    title="Top Animes (Score) for Genre: Action and Adventure",
    xaxis_title="Score",
    yaxis_title="Title",
    xaxis=dict(range=[8, 10])  # Set the range for x-axis
)
fig.write_html("../../data/plots/top_animes_score.html")
# Show the plot
fig.show()


In [38]:
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Create a subplot
fig = make_subplots()

# Group data by category
categories = top_popularity_final['category'].unique()

# Add a trace for each anime title, making bars horizontal
# Start with the first category visible
first = True
for category in categories:
    category_df = top_popularity_final[top_popularity_final['category'] == category]
    category_df.sort_values(by='popularity', inplace=True)
    fig.add_trace(
        go.Bar(y=category_df['title'], x=category_df['popularity'], name=category, 
               orientation='h', visible=first, marker_color='#ccb716')  # Set bar color to crimson
    )
    first = False  # Set the rest of the categories to invisible

# Create a dropdown menu
buttons = []
for i, category in enumerate(categories):
    button = dict(
        label=category,
        method="update",
        args=[{"visible": [i == j for j in range(len(categories))]},  # Only the selected category is visible
              {"title": f"Top Animes (Popularity) for Genre: {category}"}])
    buttons.append(button)

# Update layout for dropdown, axes, and xaxis range
fig.update_layout(
    updatemenus=[dict(
        active=0,  # First category is active
        buttons=buttons,
        direction="down",
        pad={"r": 10, "t": 10},
        showactive=True,
        x=0.77,
        xanchor="left",
        y=1.25,
        yanchor="top"
    )],
    title="Top Animes (Popularity) for Genre: Action and Adventure",
    xaxis_title="Score",
    yaxis_title="Title",
    xaxis=dict(range=[13500, 16500])  # Set the range for x-axis
)

fig.write_html("../../data/plots/top_animes_popularity.html")

# Show the plot
fig.show()
